In [1]:
# RNN으로 MNIST 문제 해결
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [3]:
#######
# 옵션 설정
#######
learning_rate = 0.001
total_epoch = 10
batch_size = 128

In [4]:
# RNN은 순서가 있는 자료를 다루므로, 한번에 입력받는 개수와, 총 몇단계로 이루어져있는 데이터를 받을지 설정
# 이를 위해 가로 픽셀수를 n_input, 세로 픽셀수를 n_step 으로 설정
n_input = 28
n_step = 28
n_hidden = 128
n_class = 10

In [5]:
#######
# 신경망 모델 구성
#######
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.float32, [None, n_class])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

# RNN 에 학습에 사용할 셀 생성
# 다음 함수를 사용하면 셀을 간단하게 변경 가능 
cell = tf.contrib.rnn.BasicRNNCell(n_hidden)

# RNN 신경망을 생성
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

# 결과를 Y: [batch_size, n_class] outputs의 형태를 이에 맞춰 변경
# outputs : [batch_size, n_step, n_hidden]
#        -> [n_step, batch_size, n_hidden]
#        -> [batch_size, n_hidden]

outputs = tf.transpose(outputs, [1,0,2])
outputs = outputs[-1]
model = tf.matmul(outputs, W)+b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [7]:
#######
# 신경망 모델 학습
#######
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(total_batch):
    total_cost = 0
    for i in range(total_batch):
        
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # X데이터를 RNN 입력 데이터에 맞게 [batch_size, n_step, n_input] 형태로 변환
        batch_xs = batch_xs.reshape((batch_size, n_step, n_input))

        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y:batch_ys})
        
        total_cost += cost_val
    
print('Epoch : ', '%04d' %(epoch+1), 'Avg cost = ', '{:.3f}'.format(total_cost / total_batch))

print('최적화 완료')

KeyboardInterrupt: 

In [ ]:
#######
# 결과 확인
#######
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

test_batch_size = len(mnist.test.images)
test_xs = mnist.test.images.reshape(test_batch_size, n_step, n_input)
test_ys = mnist.test.labels

print('정확도 : ', sess.run(accuracy, feed_dict={X: test_xs, Y: test_ys}))